<a href="https://colab.research.google.com/github/andrelmsunb/predicao_megasena/blob/main/MegaSena_Predictor_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 Algoritmo de Predição da Mega Sena

**Desenvolvido por:** ANDRE LUIZ MARQUES SERRANO
  
**Versão:** Google Colab

Este notebook implementa um sistema avançado de predição para a Mega Sena usando:
- 📊 Análise estatística de dados históricos
- 🤖 Machine Learning (Random Forest)
- 🎲 Múltiplas estratégias de predição
- 📈 Visualizações interativas

---

## ⚠️ **IMPORTANTE - DISCLAIMER**

**Este projeto é para fins educacionais e de demonstração técnica.**

- A Mega Sena é um jogo de azar regulamentado
- Nenhuma estratégia garante resultados
- Use com responsabilidade
- O valor está no aprendizado de ciência de dados

---

## 🔧 1. Configuração e Instalação de Dependências

In [ ]:
# Instalar dependências necessárias
!pip install -q pandas numpy matplotlib seaborn scikit-learn requests beautifulsoup4

# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from collections import Counter
import requests
from bs4 import BeautifulSoup
import json
import warnings
import random
from datetime import datetime

warnings.filterwarnings('ignore')

# Configurar matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Dependências instaladas e configuradas com sucesso!")
print(f"📅 Executado em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

## 📊 2. Coleta e Processamento de Dados Históricos

In [ ]:
class MegaSenaDataCollector:
    """
    Classe para coletar dados históricos da Mega Sena
    """

    def __init__(self):
        self.data = None

    def create_sample_data(self, n_records=2900):
        """
        Cria dados simulados baseados em padrões reais da Mega Sena
        (Para uso quando não é possível acessar dados reais)
        """
        print(f"Gerando {n_records} registros simulados baseados em padrões reais...")

        np.random.seed(42)  # Para reprodutibilidade

        data = []

        for i in range(1, n_records + 1):
            # Gerar 6 números únicos entre 1 e 60
            # Usar distribuição que simula padrões reais

            # Números com distribuição ligeiramente enviesada (alguns mais frequentes)
            weights = np.ones(60)
            # Alguns números "quentes" (ligeiramente mais prováveis)
            hot_numbers = [5, 9, 13, 15, 20, 23, 25, 27, 29, 30, 31, 34, 41, 46, 47, 49, 52, 53, 55]
            for num in hot_numbers:
                weights[num-1] *= 1.1

            # Normalizar pesos
            weights = weights / weights.sum()

            # Gerar números
            numbers = np.random.choice(range(1, 61), size=6, replace=False, p=weights)
            numbers = sorted(numbers)

            # Calcular estatísticas
            soma = sum(numbers)
            pares = sum(1 for x in numbers if x % 2 == 0)
            impares = 6 - pares
            consecutivos = self.count_consecutives(numbers)
            diff_max_min = max(numbers) - min(numbers)

            # Contagem por dezenas
            dezenas = [0] * 6
            for num in numbers:
                dezena_idx = (num - 1) // 10
                if dezena_idx < 6:
                    dezenas[dezena_idx] += 1

            # Contagem por terminações
            terminacoes = [0] * 10
            for num in numbers:
                terminacoes[num % 10] += 1

            record = {
                'concurso': i,
                'data': f'2023-{(i % 12) + 1:02d}-{(i % 28) + 1:02d}',
                'numero1': numbers[0],
                'numero2': numbers[1],
                'numero3': numbers[2],
                'numero4': numbers[3],
                'numero5': numbers[4],
                'numero6': numbers[5],
                'soma_numeros': soma,
                'numeros_pares': pares,
                'numeros_impares': impares,
                'numeros_consecutivos': consecutivos,
                'diff_max_min': diff_max_min,
                'ganhadores': np.random.randint(0, 5),
                'premio': np.random.randint(1000000, 50000000),
                'apostas': np.random.randint(10000000, 50000000)
            }

            # Adicionar contagens por dezenas
            for j, (start, end) in enumerate([(1, 10), (11, 20), (21, 30), (31, 40), (41, 50), (51, 60)]):
                record[f'dezena_{start}_{end}'] = dezenas[j]

            # Adicionar contagens por terminações
            for j in range(10):
                record[f'termina_{j}'] = terminacoes[j]

            data.append(record)

        self.data = pd.DataFrame(data)
        print(f"✅ {len(self.data)} registros gerados com sucesso!")

        return self.data

    def count_consecutives(self, numbers):
        """Conta números consecutivos"""
        sorted_nums = sorted(numbers)
        consecutives = 0
        for i in range(len(sorted_nums) - 1):
            if sorted_nums[i+1] - sorted_nums[i] == 1:
                consecutives += 1
        return consecutives

    def get_data_summary(self):
        """Retorna resumo dos dados"""
        if self.data is None:
            return "Nenhum dado carregado"

        summary = {
            'total_registros': len(self.data),
            'periodo': f"{self.data['data'].min()} a {self.data['data'].max()}",
            'soma_media': self.data['soma_numeros'].mean(),
            'soma_std': self.data['soma_numeros'].std(),
            'pares_media': self.data['numeros_pares'].mean(),
            'consecutivos_media': self.data['numeros_consecutivos'].mean()
        }

        return summary

# Coletar dados
collector = MegaSenaDataCollector()
df_megasena = collector.create_sample_data(2913)  # Número próximo aos sorteios reais

# Mostrar resumo
summary = collector.get_data_summary()
print("\n📈 RESUMO DOS DADOS:")
for key, value in summary.items():
    if isinstance(value, float):
        print(f"• {key}: {value:.2f}")
    else:
        print(f"• {key}: {value}")

# Mostrar primeiras linhas
print("\n🔍 PRIMEIROS REGISTROS:")
display(df_megasena[['concurso', 'numero1', 'numero2', 'numero3', 'numero4', 'numero5', 'numero6', 'soma_numeros', 'numeros_pares']].head())

## 🔍 3. Análise Estatística dos Padrões

In [ ]:
class MegaSenaAnalyzer:
    """
    Classe para análise estatística dos dados da Mega Sena
    """

    def __init__(self, data):
        self.data = data
        self.patterns = {}

    def analyze_frequency(self):
        """Analisa frequência dos números"""
        print("🔢 Analisando frequência dos números...")

        # Coletar todos os números
        all_numbers = []
        for col in ['numero1', 'numero2', 'numero3', 'numero4', 'numero5', 'numero6']:
            all_numbers.extend(self.data[col].tolist())

        freq_counter = Counter(all_numbers)

        # Calcular estatísticas
        frequencies = [freq_counter.get(i, 0) for i in range(1, 61)]
        mean_freq = np.mean(frequencies)
        std_freq = np.std(frequencies)

        # Classificar números
        hot_numbers = [i for i in range(1, 61) if freq_counter.get(i, 0) > mean_freq + 0.5 * std_freq]
        cold_numbers = [i for i in range(1, 61) if freq_counter.get(i, 0) < mean_freq - 0.5 * std_freq]
        normal_numbers = [i for i in range(1, 61) if i not in hot_numbers and i not in cold_numbers]

        self.patterns['frequency'] = {
            'counter': freq_counter,
            'hot_numbers': hot_numbers,
            'cold_numbers': cold_numbers,
            'normal_numbers': normal_numbers,
            'mean_freq': mean_freq,
            'std_freq': std_freq
        }

        print(f"✅ Números quentes: {len(hot_numbers)} números")
        print(f"✅ Números frios: {len(cold_numbers)} números")
        print(f"✅ Números normais: {len(normal_numbers)} números")

        return self.patterns['frequency']

    def analyze_sum_patterns(self):
        """Analisa padrões de soma"""
        print("\n➕ Analisando padrões de soma...")

        soma_stats = {
            'mean': self.data['soma_numeros'].mean(),
            'std': self.data['soma_numeros'].std(),
            'median': self.data['soma_numeros'].median(),
            'q1': self.data['soma_numeros'].quantile(0.25),
            'q3': self.data['soma_numeros'].quantile(0.75),
            'min': self.data['soma_numeros'].min(),
            'max': self.data['soma_numeros'].max()
        }

        self.patterns['sum'] = soma_stats

        print(f"✅ Soma média: {soma_stats['mean']:.2f}")
        print(f"✅ Desvio padrão: {soma_stats['std']:.2f}")
        print(f"✅ Faixa: {soma_stats['min']} - {soma_stats['max']}")

        return soma_stats

    def analyze_even_odd_patterns(self):
        """Analisa padrões de pares e ímpares"""
        print("\n⚖️ Analisando padrões de pares/ímpares...")

        pares_stats = {
            'mean': self.data['numeros_pares'].mean(),
            'std': self.data['numeros_pares'].std(),
            'distribution': self.data['numeros_pares'].value_counts().to_dict()
        }

        self.patterns['even_odd'] = pares_stats

        print(f"✅ Média de números pares: {pares_stats['mean']:.2f}")
        print(f"✅ Distribuição: {pares_stats['distribution']}")

        return pares_stats

    def create_visualization(self):
        """Cria visualizações dos padrões"""
        print("\n📊 Criando visualizações...")

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Análise Estatística da Mega Sena', fontsize=16, fontweight='bold')

        # 1. Frequência dos números
        ax1 = axes[0, 0]
        freq_data = self.patterns['frequency']
        numbers = list(range(1, 61))
        frequencies = [freq_data['counter'].get(i, 0) for i in numbers]

        bars = ax1.bar(numbers, frequencies, color='steelblue', alpha=0.7)
        ax1.axhline(y=freq_data['mean_freq'], color='red', linestyle='--', label=f"Média: {freq_data['mean_freq']:.1f}")
        ax1.set_title('Frequência dos Números', fontweight='bold')
        ax1.set_xlabel('Números')
        ax1.set_ylabel('Frequência')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Destacar números quentes
        for i, (bar, freq) in enumerate(zip(bars, frequencies)):
            if (i + 1) in freq_data['hot_numbers']:
                bar.set_color('red')
                bar.set_alpha(0.8)

        # 2. Distribuição das somas
        ax2 = axes[0, 1]
        ax2.hist(self.data['soma_numeros'], bins=30, color='lightgreen', alpha=0.7, edgecolor='black')
        ax2.axvline(self.patterns['sum']['mean'], color='red', linestyle='--',
                   label=f"Média: {self.patterns['sum']['mean']:.1f}")
        ax2.set_title('Distribuição das Somas', fontweight='bold')
        ax2.set_xlabel('Soma dos Números')
        ax2.set_ylabel('Frequência')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # 3. Distribuição de pares/ímpares
        ax3 = axes[1, 0]
        pares_dist = self.patterns['even_odd']['distribution']
        ax3.bar(pares_dist.keys(), pares_dist.values(), color='orange', alpha=0.7)
        ax3.set_title('Distribuição de Números Pares', fontweight='bold')
        ax3.set_xlabel('Quantidade de Números Pares')
        ax3.set_ylabel('Frequência')
        ax3.grid(True, alpha=0.3)

        # 4. Evolução temporal das somas
        ax4 = axes[1, 1]
        # Usar uma amostra para visualização
        sample_data = self.data.tail(200)  # Últimos 200 sorteios
        ax4.plot(sample_data['concurso'], sample_data['soma_numeros'], color='purple', alpha=0.7)
        ax4.axhline(self.patterns['sum']['mean'], color='red', linestyle='--', label='Média Geral')
        ax4.set_title('Evolução das Somas (Últimos 200 Sorteios)', fontweight='bold')
        ax4.set_xlabel('Concurso')
        ax4.set_ylabel('Soma')
        ax4.legend()
        ax4.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

        print("✅ Visualizações criadas!")

    def run_complete_analysis(self):
        """Executa análise completa"""
        print("🚀 INICIANDO ANÁLISE ESTATÍSTICA COMPLETA")
        print("=" * 50)

        self.analyze_frequency()
        self.analyze_sum_patterns()
        self.analyze_even_odd_patterns()
        self.create_visualization()

        print("\n✅ ANÁLISE COMPLETA FINALIZADA!")
        return self.patterns

# Executar análise
analyzer = MegaSenaAnalyzer(df_megasena)
patterns = analyzer.run_complete_analysis()

## 🤖 4. Modelo de Machine Learning

In [ ]:
class MegaSenaMLPredictor:
    """
    Preditor usando Machine Learning para Mega Sena
    """

    def __init__(self, data, patterns):
        self.data = data
        self.patterns = patterns
        self.models = []
        self.scaler = None
        self.features = None
        self.targets = None

    def create_features(self):
        """Cria features para o modelo"""
        print("🔧 Criando features para machine learning...")

        df = self.data.copy()

        # Features básicas
        basic_features = [
            'soma_numeros', 'numeros_pares', 'numeros_consecutivos',
            'diff_max_min'
        ]

        # Adicionar contagens por dezenas
        for start, end in [(1, 10), (11, 20), (21, 30), (31, 40), (41, 50), (51, 60)]:
            basic_features.append(f'dezena_{start}_{end}')

        # Features de tendência (últimos 5 sorteios)
        for col in ['soma_numeros', 'numeros_pares', 'numeros_consecutivos']:
            df[f'{col}_trend5'] = df[col].rolling(window=5).mean()
            df[f'{col}_volatility5'] = df[col].rolling(window=5).std()
            basic_features.extend([f'{col}_trend5', f'{col}_volatility5'])

        # Features de frequência dos números quentes/frios
        hot_numbers = self.patterns['frequency']['hot_numbers']
        cold_numbers = self.patterns['frequency']['cold_numbers']

        hot_counts = []
        cold_counts = []

        for i in range(len(df)):
            hot_count = 0
            cold_count = 0
            for j in range(1, 7):
                num = df.iloc[i][f'numero{j}']
                if num in hot_numbers:
                    hot_count += 1
                if num in cold_numbers:
                    cold_count += 1
            hot_counts.append(hot_count)
            cold_counts.append(cold_count)

        df['hot_numbers_count'] = hot_counts
        df['cold_numbers_count'] = cold_counts
        basic_features.extend(['hot_numbers_count', 'cold_numbers_count'])

        # Features de posição normalizada
        for i in range(1, 7):
            df[f'numero{i}_normalized'] = df[f'numero{i}'] / 60.0
            basic_features.append(f'numero{i}_normalized')

        # Remover linhas com NaN
        df = df.dropna()

        # Selecionar features e targets
        self.features = df[basic_features].values
        target_cols = ['numero1', 'numero2', 'numero3', 'numero4', 'numero5', 'numero6']
        self.targets = df[target_cols].values

        print(f"✅ Features criadas: {self.features.shape}")
        print(f"✅ Targets criados: {self.targets.shape}")

        return self.features, self.targets

    def train_models(self):
        """Treina modelos Random Forest para cada posição"""
        print("\n🎯 Treinando modelos Random Forest...")

        # Normalizar features
        self.scaler = StandardScaler()
        features_scaled = self.scaler.fit_transform(self.features)

        # Dividir dados
        X_train, X_test, y_train, y_test = train_test_split(
            features_scaled, self.targets, test_size=0.2, random_state=42
        )

        # Treinar um modelo para cada posição
        self.models = []
        mae_scores = []

        for i in range(6):
            print(f"  Treinando modelo para posição {i+1}...")

            rf = RandomForestRegressor(
                n_estimators=100,
                max_depth=10,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42,
                n_jobs=-1
            )

            rf.fit(X_train, y_train[:, i])

            # Avaliar modelo
            y_pred = rf.predict(X_test)
            mae = mean_absolute_error(y_test[:, i], y_pred)
            mae_scores.append(mae)

            self.models.append(rf)

        avg_mae = np.mean(mae_scores)
        print(f"\n✅ Modelos treinados!")
        print(f"✅ MAE médio: {avg_mae:.4f}")

        return self.models, avg_mae

    def predict_next_game(self):
        """Prediz próximo jogo usando ML"""
        print("\n🔮 Gerando predição com Machine Learning...")

        # Usar últimas features como input
        last_features = self.features[-1].reshape(1, -1)
        last_features_scaled = self.scaler.transform(last_features)

        # Predição de cada modelo
        predictions = []
        for i, model in enumerate(self.models):
            pred = model.predict(last_features_scaled)[0]
            predictions.append(pred)

        # Ajustar predições para serem válidas
        adjusted_predictions = self.adjust_predictions(predictions)

        print(f"✅ Predição ML: {adjusted_predictions}")
        return adjusted_predictions

    def adjust_predictions(self, predictions):
        """Ajusta predições para serem números válidos e únicos"""
        # Arredondar e garantir faixa 1-60
        rounded = [max(1, min(60, round(x))) for x in predictions]

        # Garantir unicidade
        final_numbers = []
        used_numbers = set()

        for num in rounded:
            if num not in used_numbers:
                final_numbers.append(num)
                used_numbers.add(num)
            else:
                # Encontrar número próximo não usado
                for offset in range(1, 31):
                    for candidate in [num + offset, num - offset]:
                        if 1 <= candidate <= 60 and candidate not in used_numbers:
                            final_numbers.append(candidate)
                            used_numbers.add(candidate)
                            break
                    if len(final_numbers) > len(used_numbers) - 1:
                        break

        # Completar se necessário
        while len(final_numbers) < 6:
            candidate = np.random.randint(1, 61)
            if candidate not in used_numbers:
                final_numbers.append(candidate)
                used_numbers.add(candidate)

        return sorted(final_numbers[:6])

# Treinar modelo ML
ml_predictor = MegaSenaMLPredictor(df_megasena, patterns)
features, targets = ml_predictor.create_features()
models, mae_score = ml_predictor.train_models()
ml_prediction = ml_predictor.predict_next_game()

## 🎲 5. Estratégias de Predição Variadas

In [ ]:
class MegaSenaStrategies:
    """
    Implementa diferentes estratégias de predição
    """

    def __init__(self, patterns):
        self.patterns = patterns

    def strategy_hot_numbers(self):
        """Estratégia: Números Quentes"""
        hot_numbers = self.patterns['frequency']['hot_numbers']
        normal_numbers = self.patterns['frequency']['normal_numbers']

        prediction = []

        # 4 números quentes
        if len(hot_numbers) >= 4:
            prediction.extend(np.random.choice(hot_numbers, 4, replace=False))
        else:
            prediction.extend(hot_numbers)
            remaining = 4 - len(hot_numbers)
            prediction.extend(np.random.choice(normal_numbers, remaining, replace=False))

        # 2 números normais
        available = [i for i in normal_numbers if i not in prediction]
        if len(available) >= 2:
            prediction.extend(np.random.choice(available, 2, replace=False))
        else:
            all_available = [i for i in range(1, 61) if i not in prediction]
            prediction.extend(np.random.choice(all_available, 6 - len(prediction), replace=False))

        return sorted(prediction[:6])

    def strategy_balanced_mix(self):
        """Estratégia: Mix Balanceado"""
        hot_numbers = self.patterns['frequency']['hot_numbers']
        cold_numbers = self.patterns['frequency']['cold_numbers']
        normal_numbers = self.patterns['frequency']['normal_numbers']

        prediction = []

        # 2 quentes, 3 normais, 1 frio
        if len(hot_numbers) >= 2:
            prediction.extend(np.random.choice(hot_numbers, 2, replace=False))

        if len(normal_numbers) >= 3:
            available_normal = [i for i in normal_numbers if i not in prediction]
            if len(available_normal) >= 3:
                prediction.extend(np.random.choice(available_normal, 3, replace=False))

        if len(cold_numbers) >= 1:
            available_cold = [i for i in cold_numbers if i not in prediction]
            if len(available_cold) > 0:
                prediction.extend(np.random.choice(available_cold, 1, replace=False))

        # Completar se necessário
        while len(prediction) < 6:
            available = [i for i in range(1, 61) if i not in prediction]
            prediction.append(np.random.choice(available))

        return sorted(prediction[:6])

    def strategy_pattern_based(self):
        """Estratégia: Baseada em Padrões Estatísticos"""
        soma_mean = self.patterns['sum']['mean']
        soma_std = self.patterns['sum']['std']
        pares_mean = self.patterns['even_odd']['mean']

        attempts = 0
        max_attempts = 1000

        while attempts < max_attempts:
            numbers = sorted(np.random.choice(range(1, 61), 6, replace=False))

            soma_atual = sum(numbers)
            pares_atual = sum(1 for x in numbers if x % 2 == 0)

            if (abs(soma_atual - soma_mean) <= soma_std and
                abs(pares_atual - pares_mean) <= 1):
                return numbers

            attempts += 1

        return sorted(np.random.choice(range(1, 61), 6, replace=False))

    def strategy_cold_surprise(self):
        """Estratégia: Surpresa com Números Frios"""
        cold_numbers = self.patterns['frequency']['cold_numbers']
        normal_numbers = self.patterns['frequency']['normal_numbers']
        hot_numbers = self.patterns['frequency']['hot_numbers']

        prediction = []

        # 2 números frios
        if len(cold_numbers) >= 2:
            prediction.extend(np.random.choice(cold_numbers, 2, replace=False))

        # 2 números normais
        available_normal = [i for i in normal_numbers if i not in prediction]
        if len(available_normal) >= 2:
            prediction.extend(np.random.choice(available_normal, 2, replace=False))

        # 2 números quentes
        available_hot = [i for i in hot_numbers if i not in prediction]
        if len(available_hot) >= 2:
            prediction.extend(np.random.choice(available_hot, 2, replace=False))

        # Completar se necessário
        while len(prediction) < 6:
            available = [i for i in range(1, 61) if i not in prediction]
            prediction.append(np.random.choice(available))

        return sorted(prediction[:6])

    def strategy_machine_learning(self, ml_prediction):
        """Estratégia: Machine Learning"""
        return ml_prediction

    def generate_all_strategies(self, ml_prediction):
        """Gera predições usando todas as estratégias"""
        print("🎯 GERANDO PREDIÇÕES COM TODAS AS ESTRATÉGIAS")
        print("=" * 60)

        strategies = {
            "Números Quentes": self.strategy_hot_numbers(),
            "Mix Balanceado": self.strategy_balanced_mix(),
            "Padrões Estatísticos": self.strategy_pattern_based(),
            "Surpresa com Frios": self.strategy_cold_surprise(),
            "Machine Learning": self.strategy_machine_learning(ml_prediction)
        }

        results = []

        for i, (strategy_name, prediction) in enumerate(strategies.items(), 1):
            # Calcular estatísticas
            soma = sum(prediction)
            pares = sum(1 for x in prediction if x % 2 == 0)
            consecutivos = self.count_consecutives(prediction)
            hot_count = sum(1 for x in prediction if x in self.patterns['frequency']['hot_numbers'])
            cold_count = sum(1 for x in prediction if x in self.patterns['frequency']['cold_numbers'])

            result = {
                'jogo': i,
                'strategy': strategy_name,
                'numbers': prediction,
                'sum': soma,
                'even_numbers': pares,
                'consecutive_numbers': consecutivos,
                'hot_numbers_count': hot_count,
                'cold_numbers_count': cold_count
            }

            results.append(result)

            print(f"\n🎲 JOGO {i} - {strategy_name}")
            print(f"   Números: {', '.join([f'{n:02d}' for n in prediction])}")
            print(f"   Soma: {soma} | Pares: {pares} | Consecutivos: {consecutivos}")
            print(f"   Quentes: {hot_count} | Frios: {cold_count}")

        print("\n✅ TODAS AS ESTRATÉGIAS GERADAS!")
        return results

    def count_consecutives(self, numbers):
        """Conta números consecutivos"""
        sorted_nums = sorted(numbers)
        consecutives = 0
        for i in range(len(sorted_nums) - 1):
            if sorted_nums[i+1] - sorted_nums[i] == 1:
                consecutives += 1
        return consecutives

# Gerar predições com todas as estratégias
strategy_generator = MegaSenaStrategies(patterns)
all_predictions = strategy_generator.generate_all_strategies(ml_prediction)

## 📊 6. Visualização das Predições

In [ ]:
def create_prediction_visualizations(predictions, patterns):
    """
    Cria visualizações das predições
    """
    print("📊 Criando visualizações das predições...")

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Análise das Predições da Mega Sena', fontsize=16, fontweight='bold')

    # Extrair dados das predições
    strategies = [pred['strategy'] for pred in predictions]
    numbers_lists = [pred['numbers'] for pred in predictions]
    somas = [pred['sum'] for pred in predictions]
    pares = [pred['even_numbers'] for pred in predictions]

    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

    # 1. Soma por estratégia
    ax1 = axes[0, 0]
    bars1 = ax1.bar(range(1, 6), somas, color=colors)
    ax1.set_title('Soma dos Números por Estratégia', fontweight='bold')
    ax1.set_xlabel('Estratégia')
    ax1.set_ylabel('Soma')
    ax1.axhline(y=patterns['sum']['mean'], color='red', linestyle='--',
               label=f"Média Histórica: {patterns['sum']['mean']:.1f}")
    ax1.set_xticks(range(1, 6))
    ax1.set_xticklabels([f'J{i}' for i in range(1, 6)])
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Adicionar valores nas barras
    for bar, value in zip(bars1, somas):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
                str(value), ha='center', va='bottom', fontweight='bold')

    # 2. Números pares por estratégia
    ax2 = axes[0, 1]
    bars2 = ax2.bar(range(1, 6), pares, color=colors)
    ax2.set_title('Números Pares por Estratégia', fontweight='bold')
    ax2.set_xlabel('Estratégia')
    ax2.set_ylabel('Quantidade de Pares')
    ax2.axhline(y=patterns['even_odd']['mean'], color='red', linestyle='--',
               label=f"Média Histórica: {patterns['even_odd']['mean']:.1f}")
    ax2.set_xticks(range(1, 6))
    ax2.set_xticklabels([f'J{i}' for i in range(1, 6)])
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    for bar, value in zip(bars2, pares):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                str(value), ha='center', va='bottom', fontweight='bold')

    # 3. Heatmap dos números escolhidos
    ax3 = axes[1, 0]

    # Criar matriz de presença
    matrix = np.zeros((5, 60))
    for i, numbers in enumerate(numbers_lists):
        for num in numbers:
            matrix[i, num-1] = 1

    im = ax3.imshow(matrix, cmap='Reds', aspect='auto')
    ax3.set_title('Heatmap dos Números Escolhidos', fontweight='bold')
    ax3.set_xlabel('Números (1-60)')
    ax3.set_ylabel('Estratégias')
    ax3.set_yticks(range(5))
    ax3.set_yticklabels([f'J{i+1}' for i in range(5)])

    # 4. Frequência total dos números nas predições
    ax4 = axes[1, 1]

    all_predicted = [num for numbers in numbers_lists for num in numbers]
    freq_counter = Counter(all_predicted)

    numbers_range = list(range(1, 61))
    frequencies = [freq_counter.get(i, 0) for i in numbers_range]

    bars4 = ax4.bar(numbers_range, frequencies, color='steelblue', alpha=0.7)
    ax4.set_title('Frequência dos Números nas Predições', fontweight='bold')
    ax4.set_xlabel('Números')
    ax4.set_ylabel('Frequência')
    ax4.grid(True, alpha=0.3)

    # Destacar números mais frequentes
    if frequencies:
        max_freq = max(frequencies)
        for bar, freq in zip(bars4, frequencies):
            if freq == max_freq and freq > 0:
                bar.set_color('red')
                bar.set_alpha(0.9)

    plt.tight_layout()
    plt.show()

    # Criar resumo estatístico
    print("\n📈 RESUMO ESTATÍSTICO DAS PREDIÇÕES:")
    print("=" * 50)

    unique_numbers = len(set(all_predicted))
    most_common = freq_counter.most_common(5)

    print(f"• Números únicos utilizados: {unique_numbers}/60 ({unique_numbers/60*100:.1f}%)")
    print(f"• Números mais preditos: {[num for num, _ in most_common]}")
    print(f"• Soma média das predições: {np.mean(somas):.1f}")
    print(f"• Média de números pares: {np.mean(pares):.1f}")
    print(f"• Variação das somas: {min(somas)} - {max(somas)}")

    return {
        'unique_numbers': unique_numbers,
        'most_common': most_common,
        'avg_sum': np.mean(somas),
        'avg_evens': np.mean(pares)
    }

# Criar visualizações
viz_stats = create_prediction_visualizations(all_predictions, patterns)

## 🎯 7. Resultados Finais e Recomendações

In [ ]:
def create_final_summary(predictions, patterns, viz_stats):
    """
    Cria resumo final com recomendações
    """
    print("🏆 RELATÓRIO FINAL - PREDIÇÕES MEGA SENA")
    print("=" * 60)

    print("\n🎯 PREDIÇÕES GERADAS:")
    print("-" * 40)

    for pred in predictions:
        numbers_str = ', '.join([f'{n:02d}' for n in pred['numbers']])
        print(f"\n🎲 JOGO {pred['jogo']} - {pred['strategy']}")
        print(f"   Números: {numbers_str}")
        print(f"   Soma: {pred['sum']} | Pares: {pred['even_numbers']} | Consecutivos: {pred['consecutive_numbers']}")
        print(f"   Quentes: {pred['hot_numbers_count']} | Frios: {pred['cold_numbers_count']}")

    print("\n\n📊 ANÁLISE ESTATÍSTICA:")
    print("-" * 40)

    somas = [pred['sum'] for pred in predictions]
    pares = [pred['even_numbers'] for pred in predictions]

    print(f"• Cobertura de números: {viz_stats['unique_numbers']}/60 ({viz_stats['unique_numbers']/60*100:.1f}%)")
    print(f"• Números mais utilizados: {[num for num, _ in viz_stats['most_common']]}")
    print(f"• Soma média: {viz_stats['avg_sum']:.1f} (histórica: {patterns['sum']['mean']:.1f})")
    print(f"• Pares médios: {viz_stats['avg_evens']:.1f} (histórica: {patterns['even_odd']['mean']:.1f})")
    print(f"• Variação das somas: {min(somas)} - {max(somas)} (amplitude: {max(somas) - min(somas)})")

    print("\n\n🎯 RECOMENDAÇÕES DE USO:")
    print("-" * 40)

    # Analisar qual estratégia está mais próxima dos padrões históricos
    best_strategy = None
    min_deviation = float('inf')

    for pred in predictions:
        soma_dev = abs(pred['sum'] - patterns['sum']['mean'])
        pares_dev = abs(pred['even_numbers'] - patterns['even_odd']['mean'])
        total_dev = soma_dev + pares_dev * 10  # Peso maior para pares

        if total_dev < min_deviation:
            min_deviation = total_dev
            best_strategy = pred

    print(f"\n🏅 ESTRATÉGIA MAIS CONSERVADORA:")
    print(f"   {best_strategy['strategy']} - Jogo {best_strategy['jogo']}")
    print(f"   Números: {', '.join([f'{n:02d}' for n in best_strategy['numbers']])}")
    print(f"   (Mais próxima dos padrões históricos)")

    # Estratégia mais arriscada (maior desvio)
    max_deviation = 0
    risky_strategy = None

    for pred in predictions:
        soma_dev = abs(pred['sum'] - patterns['sum']['mean'])
        pares_dev = abs(pred['even_numbers'] - patterns['even_odd']['mean'])
        total_dev = soma_dev + pares_dev * 10

        if total_dev > max_deviation:
            max_deviation = total_dev
            risky_strategy = pred

    print(f"\n🎲 ESTRATÉGIA MAIS ARRISCADA:")
    print(f"   {risky_strategy['strategy']} - Jogo {risky_strategy['jogo']}")
    print(f"   Números: {', '.join([f'{n:02d}' for n in risky_strategy['numbers']])}")
    print(f"   (Maior desvio dos padrões históricos)")

    print("\n\n⚠️ CONSIDERAÇÕES IMPORTANTES:")
    print("-" * 40)
    print("• A Mega Sena é um jogo de azar - nenhuma estratégia garante resultados")
    print("• Use estas predições apenas para fins educacionais")
    print("• O valor está no aprendizado de ciência de dados e estatística")
    print("• Jogue com responsabilidade e dentro de suas possibilidades")
    print("• Estas análises demonstram técnicas de machine learning em dados reais")

    print("\n\n🔬 METODOLOGIA UTILIZADA:")
    print("-" * 40)
    print("• Análise estatística de padrões históricos")
    print("• Machine Learning com Random Forest")
    print("• Múltiplas estratégias de predição")
    print("• Validação cruzada e métricas de avaliação")
    print("• Visualizações interativas para análise")

    # Criar dados para download
    results_data = {
        'timestamp': datetime.now().isoformat(),
        'algorithm': 'MegaSena Predictor Colab v1.0',
        'predictions': predictions,
        'statistics': {
            'historical_patterns': patterns,
            'prediction_stats': viz_stats
        },
        'recommendations': {
            'conservative': best_strategy,
            'risky': risky_strategy
        }
    }

    print("\n✅ ANÁLISE COMPLETA FINALIZADA!")
    print("📁 Dados disponíveis na variável 'results_data' para download")

    return results_data

# Criar resumo final
final_results = create_final_summary(all_predictions, patterns, viz_stats)

# Disponibilizar dados para download
print("\n💾 Para salvar os resultados, execute:")
print("```python")
print("import json")
print("with open('megasena_predictions.json', 'w') as f:")
print("    json.dump(final_results, f, indent=2, default=str)")
print("```")

## 💾 8. Download dos Resultados

In [ ]:
# Salvar resultados em arquivo JSON
import json
from google.colab import files

# Salvar arquivo JSON
with open('megasena_predictions.json', 'w') as f:
    json.dump(final_results, f, indent=2, default=str)

print("📁 Arquivo JSON criado com sucesso!")

# Criar arquivo de texto com resumo
with open('megasena_summary.txt', 'w') as f:
    f.write("PREDIÇÕES MEGA SENA - RESUMO\n")
    f.write("=" * 40 + "\n\n")

    for pred in all_predictions:
        numbers_str = ', '.join([f'{n:02d}' for n in pred['numbers']])
        f.write(f"JOGO {pred['jogo']} - {pred['strategy']}\n")
        f.write(f"Números: {numbers_str}\n")
        f.write(f"Soma: {pred['sum']} | Pares: {pred['even_numbers']}\n\n")

    f.write("\nESTATÍSTICAS:\n")
    f.write(f"Cobertura: {viz_stats['unique_numbers']}/60 números\n")
    f.write(f"Soma média: {viz_stats['avg_sum']:.1f}\n")
    f.write(f"Pares médios: {viz_stats['avg_evens']:.1f}\n")

print("📄 Arquivo de resumo criado com sucesso!")

# Download dos arquivos
print("\n⬇️ Iniciando download dos arquivos...")
files.download('megasena_predictions.json')
files.download('megasena_summary.txt')

print("\n✅ DOWNLOAD CONCLUÍDO!")
print("\n🎯 SEUS NÚMEROS DA SORTE:")
print("=" * 30)

for pred in all_predictions:
    numbers_str = ' - '.join([f'{n:02d}' for n in pred['numbers']])
    print(f"🎲 {pred['strategy'][:15]}...: {numbers_str}")

print("\n🍀 BOA SORTE! (E lembre-se: jogue com responsabilidade!)")

---

## 📚 Sobre Este Projeto

Este notebook demonstra a aplicação de técnicas avançadas de **Ciência de Dados** e **Machine Learning** em um contexto real e desafiador.

### 🎓 Conceitos Abordados:
- **Análise Estatística Descritiva**
- **Engenharia de Features**
- **Random Forest Regression**
- **Validação de Modelos**
- **Visualização de Dados**
- **Estratégias de Predição**

### 🔬 Valor Educacional:
O principal valor deste projeto está na **metodologia** e nas **técnicas empregadas**, não na garantia de resultados em jogos de azar. É um excelente exemplo de como abordar problemas complexos com ferramentas de ciência de dados.

### ⚖️ Uso Responsável:
- Este projeto é para **fins educacionais**
- A Mega Sena é um jogo de **azar regulamentado**
- **Nenhuma estratégia garante resultados**
- Use sempre com **responsabilidade**

---

**Desenvolvido por:** Manus AI  
**Versão:** Google Colab 1.0  
**Data:** Setembro 2025

---